### Final Data Cleaning

In this notebook I shall write down a step by step data cleaning process from all the insights gathered from the previuos notebooks.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
#import
import os, sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

In [3]:
module_path = os.path.abspath(os.path.join(os.pardir, os.pardir))
if module_path not in sys.path:
    sys.path.append(module_path)

In [4]:
#import customized functions
from src.data_cleaning import cleaning_functions as cfs
from src.data_cleaning import exploration_functions as efs
from src.data_cleaning import processing_functions as pfs

#### First Load data and make Train Test splits

In [5]:
def load_data_and_split():
    """In this function we use the Pandas read_csv method to 
    convert the csv files into dataframes. These csv files are 
    saved in hte data folder as:
    >>> 'training_set_values.csv' (for X features)
    >>> 'training_set_labels.csv' (for y target)
    
    We then split the dataframes into training and testing sets 
    and return the X_train, X_test, y_train, y_test
    """
    
    #read csvs
    import pandas as pd
    features = pd.read_csv('../../data/training_set_values.csv')
    labels = pd.read_csv('../../data/training_set_labels.csv')
    
    #train and test split, random_state of 2020, test_size = 25%
    from sklearn.model_selection import train_test_split
    X_train, X_test, y_train, y_test = train_test_split(features, labels, random_state=2020, test_size=0.25) 
    
    return X_train, X_test, y_train, y_test

X_train, X_test, y_train, y_test = load_data_and_split()

#### Trim down X_train and X_test

#### Identified columns that we will be dropping:

`id`, `date_recorded`, `recorded_by`, `wpt_name`, `scheme_name`, `num_private`, `subvillage`,
`ward`, `longitude`, `latitude`, `extraction_type_class`, `management_group`, `payment_type`, 
`quality_group`, `quantity_group`, `source_type`, `source_class`, `waterpoint_type_group`, `installer`, `funder`

In [6]:
def drop_unnecessary_feature_columns(df):
    """This function drops all the investigated unnecessary 
    columns from the features dataframe and returns the 
    trimmed datadrame.
    """
    
    df.drop(['id', 'date_recorded', 'recorded_by', 'wpt_name',
             'scheme_name', 'num_private', 'subvillage', 'ward',
             'longitude', 'latitude', 'extraction_type_class', 
             'management_group', 'payment_type', 'quality_group',
             'quantity_group', 'source_type', 'source_class', 
             'waterpoint_type_group', 'installer', 'funder'], 
              axis=1, inplace=True)
    
    return df

X_train = drop_unnecessary_feature_columns(X_train)
X_test = drop_unnecessary_feature_columns(X_test)

#### Fill NaNs 

In [7]:
cfs.fill_all_nans(X_train)
cfs.fill_all_nans(X_test)

#### Convert y_train and y_test `status_group` to classes

In [8]:
#first drop id
def drop_id_from_y(df):
    """This function drops id column 
    from the y sets
    """
    df.drop('id', axis=1, inplace=True)
    
    return df


#convert `status_group` to classes using LabelEncoder
def create_class_labels_for_y(df1, df2):
    """This function takes the y training set as df1
    and y test set as df2 and creates labels 
    for the target.
    Here we fit on training and transform both the 
    training and the test set
    Returns: training df, test df and label dictionary.
    """
    
    le = LabelEncoder()
    le.fit(df1['status_group'])
    target1 = le.transform(df1['status_group'])
    df1 = pd.DataFrame(target1, columns=['target'])
    
    target2 = le.transform(df2['status_group'])
    df2 = pd.DataFrame(target2, columns=['target'])
    
    
    classes_dict = {k:v for k,v in zip(
                            le.transform(['functional', 'functional needs repair', 'non functional']), 
                            ['functional', 'functional needs repair', 'non functional'])}
    
    return df1, df2, classes_dict

def process_y_sets(df1, df2):
    """This function preprocess the y_train and y_test
    by dropping the id column and converting the status_group
    to classes 
    
    Returns: y_train, y_test and label dictionary.
    """
    
    #helper functio to drop id
    df1 = drop_id_from_y(df1)
    df2 = drop_id_from_y(df2)
    
    #helper function to make class labels
    y_train, y_test, classes_dict = create_class_labels_for_y(df1, df2)
    
    return y_train, y_test, classes_dict

In [9]:
y_train, y_test, classes_dict = process_y_sets(y_train, y_test)

In [10]:
y_train.head(5)

,target
0,0
1,0
2,0
3,2
4,0


In [11]:
classes_dict    

{0: 'functional', 1: 'functional needs repair', 2: 'non functional'}

##### Combining all functions here...

In [12]:
def processed_dataset():
    """This function loads the two datasets from the data folder
    and splits it up into X_train, X_test, y_train, y_test 
    with random_state = 2020, test_size = 25%
    
    Then it trims down X Features and create y labels
    
    Returns: X_train, X_test, y_train, y_test, classes_dict
    """
    #using helper function to split dataset
    X_train, X_test, y_train, y_test = load_data_and_split()
    
    #using helper function to trim down X features
    X_train = drop_unnecessary_feature_columns(X_train)
    X_test = drop_unnecessary_feature_columns(X_test)
    
    #using helper functio to fill NaNs
    fill_all_nans(X_train)
    fill_all_nans(X_test)
    
    #using helper function to process y_train and y_test
    y_train, y_test, classes_dict = process_y_sets(y_train, y_test)
    
    return X_train, X_test, y_train, y_test, classes_dict


In [13]:
X_train, X_test, y_train, y_test, classes_dict = pfs.processed_dataset()

In [14]:
classes_dict

{0: 'functional', 1: 'functional needs repair', 2: 'non functional'}

In [15]:
X_train.dtypes

amount_tsh               float64
gps_height                 int64
basin                     object
region                    object
region_code                int64
district_code              int64
lga                       object
population                 int64
public_meeting            object
scheme_management         object
permit                    object
construction_year          int64
extraction_type           object
extraction_type_group     object
management                object
payment                   object
water_quality             object
quantity                  object
source                    object
waterpoint_type           object
dtype: object

In [16]:
def identifying_feature_types(df):
    """This function lists out the names of columns
    in a dataframe whose dtypes are objects as categorical
    features and the numeric types as numeric features
    
    Returns two lists of columns names first for numeric 
    and the second for categorical features
    """
    
    numeric_features = []
    categorical_features = []
    for name, dtype in zip(df.dtypes.index, df.dtypes):
        if str(dtype).startswith('ob'):
            categorical_features.append(name)
        else:
            numeric_features.append(name)
    
    return numeric_features, categorical_features

In [17]:
num_feats, cat_feats = identifying_feature_types(X_train)

In [18]:
num_feats

['amount_tsh',
 'gps_height',
 'region_code',
 'district_code',
 'population',
 'construction_year']

In [19]:
cat_feats

['basin',
 'region',
 'lga',
 'public_meeting',
 'scheme_management',
 'permit',
 'extraction_type',
 'extraction_type_group',
 'management',
 'payment',
 'water_quality',
 'quantity',
 'source',
 'waterpoint_type']

In [20]:
def one_hot_encode_feature(df, name):
    """This funciton takes in a dataframe and a feature name and 
    One hot encodes the feature and adds it to the dataframe
    
    Returns transformed dataframe and the ohe object 
    used to transform the frame
    """
    
    ohe = OneHotEncoder(categories='auto', handle_unknown='ignore')
    single_feature_df = df[[name]]
    ohe.fit(single_feature_df)
    feature_array = ohe.transform(single_feature_df).toarray()
    ohe_df = pd.DataFrame(feature_array, columns=ohe.categories_[0], index=df.index)
    df = df.drop(name, axis=1)
    df = pd.concat([df, ohe_df], axis=1)
    
    #returning ohe here so that it can be used to transform X_test later
    return df, ohe

#can use this function in a loop to convert all the categorical features into encoded features!

#### Let's encode all the categorical features and add them to the dataframe

In [21]:
def ohe_all_categorical_features(df):
    """This function takes in a dataframe, identifies the
    dtypes in the dataframe and uses the object dtypes to
    list out categorical columns
    
    Next it use OneHotEncoder to convert those Categorical 
    features
    
    Returns: the transformed dataframe and a dictionary 
    containing the ohe object that can be used later to 
    transform the testing dataset
    """
    
    
    #helper function to identify categorical feature names
    num_feats, cat_feats = identifying_feature_types(df)
    
    #reassuring the values in categorical features are str types
    for name in cat_feats:
        df[name] = df[name].astype(str)
    
    #use helper function in loop to transform dataframe
    encoders = {}
    
    for name in cat_feats:
        df, ohe = one_hot_encode_feature(df, name)
        encoders[name] = ohe
    
    return df, encoders

Checking to see if this worked....

In [22]:
X_train, encoders = ohe_all_categorical_features(X_train)

In [23]:
X_train

,amount_tsh,gps_height,region_code,district_code,population,construction_year,Internal,Lake Nyasa,Lake Rukwa,Lake Tanganyika,...,shallow well,spring,unknown,cattle trough,communal standpipe,communal standpipe multiple,dam,hand pump,improved spring,other
29193,0.0,0,19,8,0,0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
35859,1200.0,1228,16,2,650,2002,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
57006,0.0,0,17,5,0,0,0.0,0.0,0.0,1.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
15959,0.0,954,21,4,200,2003,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
21582,0.0,0,12,4,0,0,0.0,1.0,0.0,0.0,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18523,0.0,1944,15,3,150,1984,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
44739,0.0,1521,15,2,100,1991,0.0,0.0,1.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
47734,0.0,0,12,6,0,0,0.0,0.0,1.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
40259,500.0,1293,10,3,60,2002,0.0,1.0,0.0,0.0,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


### That worked!!!

## Next Steps:

* Start with the modeling
* Finalize the metric/metrics that are important to answer our business question
* Look up ensemble methods 